In [1]:
from datetime import datetime, timedelta

alerts = {}

timestamps = {}
time_format = "%H:%M:%S"

def parse_time(time_str):
    return datetime.strptime(time_str, time_format)

def is_within_timeframe(time1, time2, seconds):
    return abs((time1 - time2).total_seconds()) <= seconds

def add_alert(alert):
    alert_id = alert["id"]
    timestamp = parse_time(alert["timestamp"])
    threat_level = alert["threat_level"]

    if alert_id in alerts:
        existing_alert = alerts[alert_id]
        existing_timestamp = timestamps[alert_id]

        if is_within_timeframe(timestamp, existing_timestamp, 30):
            if threat_level > existing_alert["threat_level"]:
                alerts[alert_id] = alert
                timestamps[alert_id] = timestamp
        elif timestamp > existing_timestamp + timedelta(minutes=5):
            alerts[alert_id] = alert
            timestamps[alert_id] = timestamp
    else:
        alerts[alert_id] = alert
        timestamps[alert_id] = timestamp

    evict_old_alerts()

def evict_old_alerts():
    current_time = max(timestamps.values(), default=datetime.min)
    evict_before = current_time - timedelta(minutes=5)
    to_evict = [alert_id for alert_id, timestamp in timestamps.items() if timestamp < evict_before]

    for alert_id in to_evict:
        del alerts[alert_id]
        del timestamps[alert_id]

def get_stored_alerts():
    return list(alerts.values())

incoming_alerts = [
    {"id": "A123", "timestamp": "00:00:10", "threat_level": 3},
    {"id": "A123", "timestamp": "00:00:15", "threat_level": 3},
    {"id": "B456", "timestamp": "00:00:20", "threat_level": 2},
    {"id": "A123", "timestamp": "00:00:30", "threat_level": 5},
    {"id": "B456", "timestamp": "00:05:05", "threat_level": 2}
]

for alert in incoming_alerts:
    add_alert(alert)

result = get_stored_alerts()
print("Stored alerts:", result)


Stored alerts: [{'id': 'A123', 'timestamp': '00:00:30', 'threat_level': 5}, {'id': 'B456', 'timestamp': '00:00:20', 'threat_level': 2}]
